# My model on `pgmpy` framework

In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [2]:
# Defining the network structure
model = BayesianNetwork(
    [
        ("debtsToIncomeRatio", "paymentHistory"),
        ("degree", "income"),
        ("income", "assets"),
        ("assets", "futureIncome"),
        ("income", "futureIncome"),
        ("creditTime", "fullPayoutProb"),
        ("age", "fullPayoutProb"),
        ("age", "reliability"),
        ("age", "paymentHistory"),
        ("paymentHistory", "reliability"),
        
        ("fullPayoutProb", "creditWorthiness"),
        ("reliability", "creditWorthiness"),
        ("futureIncome", "creditWorthiness"),
        ("debtsToIncomeRatio", "creditWorthiness"),
    ]
)

## Defining the CPDs

In [3]:
# Defining the CPD for credit_time:
cpd_cre_time = TabularCPD("creditTime", 2, [[0.4], [0.6]])

In [4]:
# Defining the CPD for debts_to_income_ratio:
cpd_dti = TabularCPD("debtsToIncomeRatio", 2, [[0.3], [0.7]])

In [5]:
# Defining the CPD for degree:
cpd_deg = TabularCPD("degree", 2, [[0.25], [0.75]])

In [6]:
# Defining the CPD for income:
cpd_inc = TabularCPD(
    "income", 
     2, 
     [
         [.2, .6],
         [.8, .4]
     ], 
     evidence=['degree'], 
     evidence_card=[2]
)

In [7]:
# Defining the CPD for assets:
cpd_ast = TabularCPD(
    "assets", 
     2, 
     [
         [.9, .4],
         [.1, .6]
     ], 
     evidence=['income'], 
     evidence_card=[2]
)

In [8]:
# Defining the CPD for future_income:
cpd_fin = TabularCPD(
    "futureIncome",
    2,
    [
        [.995, .6, .7, .1],
        [.005, .4, .3, .9],
    ],
    evidence=["assets", "income"],
    evidence_card=[2, 2],
)

In [9]:
# Defining the CPD for age:
cpd_age = TabularCPD("age", 2, [[0.8], [0.2]])

In [10]:
# Defining the CPD for payment_history:
cpd_phs = TabularCPD(
    "paymentHistory",
    2,
    [
        [.2, .3, .8, .9],
        [.8, .7, .2, .1],
    ],
    evidence=["debtsToIncomeRatio", "age"],
    evidence_card=[2, 2],
)

In [11]:
# Defining the CPD for reliability:
cpd_rel = TabularCPD(
    "reliability",
    2,
    [
        [.8, .4, .95, .2],
        [.2, .6, .05, .8],
    ],
    evidence=["age", "paymentHistory"],
    evidence_card=[2, 2],
)

In [12]:
# Defining the CPD for full_payout_probability:
cpd_fpp = TabularCPD(
    "fullPayoutProb",
    2,
    [
        [.95, .9, .9, .3],
        [.05, .1, .1, .7],
    ],
    evidence=["creditTime", "age"],
    evidence_card=[2, 2],
)

In [13]:
# Defining the CPDs for credit_worthiness:
cpd_cw = TabularCPD(
    "creditWorthiness",
    2,
    [
        [.7, .6, .35, .5, .999, .9, .75, .55, .5, .05, .2, .0001, .7, .6, .1, .003],
        [.3, .4, .65, .5, .001, .1, .25, .45, .5, .95, .8, .9999, .3, .4, .9, .997],
    ],
    evidence=["futureIncome", "debtsToIncomeRatio", "fullPayoutProb", "reliability"],
    evidence_card=[2, 2, 2, 2],
)

In [14]:
# Associating the CPDs with the network structure.
model.add_cpds(
    cpd_cre_time,
    cpd_dti,
    cpd_deg,
    cpd_inc,
    cpd_ast,
    cpd_fin,
    cpd_age,
    cpd_phs,
    cpd_rel,
    cpd_fpp,
    cpd_cw
)

In [15]:
# Checking the model for various errors
model.check_model()

True

## Using the model

In [16]:
infer = VariableElimination(model)
credit_result = infer.query(['creditWorthiness'])

In [17]:
print(credit_result)

+---------------------+-------------------------+
| creditWorthiness    |   phi(creditWorthiness) |
+=====================+=========================+
| creditWorthiness(0) |                  0.7051 |
+---------------------+-------------------------+
| creditWorthiness(1) |                  0.2949 |
+---------------------+-------------------------+
